<a href="https://colab.research.google.com/github/dvircohen0/Machine-Learning-Algorithms-From-Scratch/blob/main/naive_bayes_spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget http://openclassroom.stanford.edu/MainFolder/courses/MachineLearning/exercises/ex6materials/ex6DataEmails.zip
!unzip /content/ex6DataEmails.zip

In [266]:
import os
import numpy as np

In [267]:
class Vocabulary():

  def __init__(self, folder_path):
    self.folder_path = folder_path
    self.word2count={}
    self.wordprob={}
    self.num_sent = 0
    self.smoothing = 1e-100
    self.path_to_list()
    self.fill_matrix()

# read file into list of words
  def file_to_word_list(self,file_path):
    a_file = open(file_path,'r')
    for line in a_file:
      stripped = line.strip()
      lower = stripped.lower()
      line_list = lower.split()
    return line_list

# read files from path
  def path_to_list(self):
    files = os.listdir(self.folder_path)
    self.list_of_lists =[]
    for file in files:
      full_path = os.path.join(self.folder_path,file)
      self.list_of_lists.append(self.file_to_word_list(full_path))

# count word
  def add_word(self, word):
      if word not in self.word2count:
          self.word2count[word] = 1
      else:
          self.word2count[word] += 1

# add words from sentence
  def add_sentence(self, sentence):
    self.num_sent +=1
    for word in sentence:
      self.add_word(word)

# return prob for word
  def word_probs(self):
    self.total_words = sum(self.word2count.values())
    for word in self.word2count:
      self.wordprob[word] = self.word2count[word]/self.total_words

# fill prob list
  def fill_matrix(self):
    for mail in self.list_of_lists:
      self.add_sentence(mail)
    self.word_probs()

# predict prob of sentence
  def predict_prob(self, file_path):
    sent = self.file_to_word_list(file_path)
    prob=0
    for word in sent:
      if word in self.wordprob:
        prob += np.log1p(self.wordprob[word])
      else: prob += self.smoothing
    return prob

In [274]:
import glob

spam = Vocabulary(folder_path ='/content/spam-train')
ham = Vocabulary(folder_path ='/content/nonspam-train')

spam_test_files = glob.glob('/content/spam-test/*.txt')
ham_test_files = glob.glob('/content/nonspam-test/*.txt')

test_files = spam_test_files + ham_test_files
y_true = np.hstack((np.zeros(len(spam_test_files)),
                    np.ones(len(ham_test_files))))

spam_prior = np.log1p(spam.num_sent/(spam.num_sent + ham.num_sent))
ham_prior = np.log1p(ham.num_sent/(spam.num_sent + ham.num_sent))


count = 0
for i in range(len(y_true)):
  spam_prob = spam.predict_prob(test_files[i]) + spam_prior
  ham_prob = ham.predict_prob(test_files[i]) + ham_prior
  prediction = np.argmax([spam_prob,ham_prob])
  if prediction == y_true[i]:
    count+=1

print('accuracy score {:.2%}'.format(count/len(y_true)))

accuracy score 90.00%
